<a href="https://colab.research.google.com/github/dtype2100/chat_bot/blob/master/fine_tuning_Llama_3_2_3B_Instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q trl peft accelerate bitsandbytes datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
# 2. 로딩
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # 꼭 지정해야 함

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # Changed device_map to "cpu" to force CPU usage
    device_map="cpu",
    # Removed load_in_4bit as it requires CUDA
    # load_in_4bit=True,
    torch_dtype="auto",
    trust_remote_code=True
)

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [10]:
from datasets import Dataset

train_data = [
    {
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "What is the capital of France?"},
            {"role": "assistant", "content": "The capital of France is Paris."}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "한국의 수도는 어디야?"},
            {"role": "assistant", "content": "서울입니다."}
        ]
    }
]

dataset = Dataset.from_list(train_data)


In [15]:
import os
os.environ["WANDB_MODE"] = "disabled"


In [16]:
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig, get_peft_model

# Configure PEFT (LoRA)
peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"  # Ensure task type is set correctly
)

# Apply PEFT to the model
model = get_peft_model(model, peft_config)

sft_config = SFTConfig(
    output_dir="./llama3.2-qlora-output",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-5,
    logging_steps=10,
    save_steps=100,
    warmup_ratio=0.1,
    max_seq_length=1024,
    # Removed: dataset_format="chat",
    # Removed: dataset_text_field="messages",  # HuggingFace Datasets field
    # Removed: template="chatml"  # ChatML 포맷에 맞게 구성된 messages 필드 사용 시
)

trainer = SFTTrainer(
    model=model,
    # Removed tokenizer argument and passed it via the args parameter in SFTConfig
    # tokenizer=tokenizer,
    train_dataset=dataset,
    args=sft_config
)

trainer.train()

Converting train dataset to ChatML:   0%|          | 0/2 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:


Abort: 

In [13]:
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig, get_peft_model

# Configure PEFT (LoRA)
peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"  # Ensure task type is set correctly
)

# Apply PEFT to the model
model = get_peft_model(model, peft_config)

sft_config = SFTConfig(
    output_dir="./llama3.2-qlora-output",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-5,
    logging_steps=10,
    save_steps=100,
    warmup_ratio=0.1,
    max_seq_length=1024,
    # Removed: dataset_format="chat",
    # Removed: dataset_text_field="messages",  # HuggingFace Datasets field
    # Removed: template="chatml"  # ChatML 포맷에 맞게 구성된 messages 필드 사용 시
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=sft_config
)

trainer.train()

TypeError: SFTTrainer.__init__() got an unexpected keyword argument 'tokenizer'

In [ ]:
trainer.model.save_pretrained("./llama3.2-qlora")
tokenizer.save_pretrained("./llama3.2-qlora")

# 텍스트 생성 예시
from transformers import pipeline

pipe = pipeline("text-generation", model="./llama3.2-qlora", tokenizer=tokenizer, device_map="auto")

print(pipe("질문: 서울의 명소는?\n답변:", max_new_tokens=100)[0]["generated_text"])
